'Context separator contains 3 tokens'

In [12]:
# Prod

In [16]:
query = "What are the main sectors of activity?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
d81b9d188f526726d84381fa7b041bff
5f02f82ee781f94541f101b8aaf64a18

Q: What are the main sectors of activity?
A: Agriculture and forestry, built environment, business and services, health and wellbeing, infrastructure, local government, and natural environment.
